In [1]:
from google.colab import drive
drive.mount('/content/drive')
!pip install deepchem


Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 608 kB 4.0 MB/s 
     |████████████████████████████████| 36.8 MB 18 kB/s 


In [2]:
import glob,os

import pandas as pd
import deepchem as dc
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw, PyMol, rdFMCS
from rdkit.Chem.Draw import IPythonConsole
from rdkit import rdBase
from deepchem import metrics
from IPython.display import Image, display
from rdkit.Chem.Draw import SimilarityMaps
import tensorflow as tf

In [ ]:
A2:A200001
A200002:A400001
A400002:A600001
A600002:A800001
A800002:A1000001

In [3]:
DATASET_FILE ='/content/drive/My Drive/DL/20220902-count372/数据库.csv'
MODEL_DIR = '/content/drive/My Drive/DL/models'


data = pd.read_csv(DATASET_FILE)
data.head()

,Smiles,Tg/℃
0,O=C1C2=C(CN1[*])C=C3C(C(N(C4=CC=CC=C4OC5=CC=C(...,386.0
1,O=C(N1CCCCCCCCN2C(C3=C([*])C=CC=C3C2=O)=O)C4=C...,128.0
2,O=C(N1CCCCCCCCN2C(C3=C([*])C=CC=C3C2=O)=O)C4=C...,135.0
3,O=C(N1C2=CC=CC=C2OC3=CC=CC=C3N4C(C5=CC=C(C([*]...,289.0
4,O=C(N1C2=CC=CC=C2OC3=CC=CC(N4C(C5=CC=C(C([*])=...,259.0


In [4]:
###########################################Two Conv.##########################################
"""
graph_conv_layers: [64,64]
dense_layer_size: 128
batch_size: 20
dropout: 0.2
nb_epoch: 1500

""" 


import time

start = time.time()

metrics_rmse_train = []
metrics_mae_train = []
metrics_r2_train = []
metrics_rmse_test = []
metrics_mae_test = []
metrics_r2_test = []

for i in range(5):   # 5次循环
  print("Executing: %d/5" %(i+1))
  print("#"*60)
  os.mkdir(MODEL_DIR + str(i+1))
  MODEL_DIR = MODEL_DIR + str(i+1)
  ########################################Featurizerization#########################
  featurizer = dc.feat.ConvMolFeaturizer()
  loader = dc.data.CSVLoader(tasks=["Tg/℃"], feature_field="Smiles", featurizer=featurizer)
  dataset = loader.create_dataset(DATASET_FILE, shard_size=10000)

  splitter = dc.splits.splitters.RandomSplitter()
  trainset, testset = splitter.train_test_split(dataset, frac_train=0.8,seed=1)

  ########################################Model######################################
  model = dc.models.GraphConvModel(1, 
                  dense_layer_size =128,
                  mode="regression",
                  batch_normalize=False,
                  batch_size=20,
                  model_dir=MODEL_DIR,
                  dropout=0.2)

  ########################################Fit########################################
  model.fit(trainset, nb_epoch=1500)

  ########################################Predict####################################
  test_pred = model.predict(testset)
  train_pred = model.predict(trainset)

  ########################################Metrics####################################
  rmse = metrics.mean_squared_error(y_true=trainset.y, y_pred=train_pred, squared=False)   # RMSE
  r2 = metrics.r2_score(y_true=trainset.y, y_pred=train_pred)
  mae = metrics.mean_absolute_error(y_true=trainset.y, y_pred=train_pred)

  rmse_test = metrics.mean_squared_error(y_true=testset.y, y_pred=test_pred, squared=False)   # RMSE
  r2_test = metrics.r2_score(y_true=testset.y, y_pred=test_pred)
  mae_test = metrics.mean_absolute_error(y_true=testset.y, y_pred=test_pred)

  metrics_r2_train.append(r2)
  metrics_rmse_train.append(rmse)
  metrics_mae_train.append(mae)
  metrics_r2_test.append(r2_test)
  metrics_rmse_test.append(rmse_test)
  metrics_mae_test.append(mae_test)

end = time.time()

print("Time cost for GNN without global node: %.3f min" % ((end-start)/60))

print("Train_R2: %.2f (+/- %.2f)" % (np.mean(metrics_r2_train), np.std(metrics_r2_train)))
print("Train_RMSE: %.2f (+/- %.2f)" % (np.mean(metrics_rmse_train), np.std(metrics_rmse_train)))
print("Train_MAE: %.2f (+/- %.2f)" % (np.mean(metrics_mae_train), np.std(metrics_mae_train)))

print("Test_R2: %.2f (+/- %.2f)" % (np.mean(metrics_r2_test), np.std(metrics_r2_test)))
print("Test_RMSE: %.2f (+/- %.2f)" % (np.mean(metrics_rmse_test), np.std(metrics_rmse_test)))
print("Test_MAE: %.2f (+/- %.2f)" % (np.mean(metrics_mae_test), np.std(metrics_mae_test)))

Executing: 1/5
############################################################


    rdkit.Chem.rdmolfiles.CanonicalRankAtoms(NoneType)
did not match C++ signature:
    CanonicalRankAtoms(RDKit::ROMol mol, bool breakTies=True, bool includeChirality=True, bool includeIsotopes=True)
/usr/local/lib/python3.7/dist-packages/deepchem/feat/base_classes.py:305: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.asarray(features)
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:446: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model/graph_pool_1/Reshape_14:0", shape=(193,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_keras_model/graph_pool_1/Reshape_13:0", shape=(193, 64), dtype=float32), dense_shape=Tensor("gradient_tape/privat

Executing: 2/5
############################################################


    rdkit.Chem.rdmolfiles.CanonicalRankAtoms(NoneType)
did not match C++ signature:
    CanonicalRankAtoms(RDKit::ROMol mol, bool breakTies=True, bool includeChirality=True, bool includeIsotopes=True)
/usr/local/lib/python3.7/dist-packages/deepchem/feat/base_classes.py:305: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.asarray(features)
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:446: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model_1/graph_pool_3/Reshape_14:0", shape=(193,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_keras_model_1/graph_pool_3/Reshape_13:0", shape=(193, 64), dtype=float32), dense_shape=Tensor("gradient_tape/pr

Executing: 3/5
############################################################


...

Executing: 4/5
############################################################


    rdkit.Chem.rdmolfiles.CanonicalRankAtoms(NoneType)
did not match C++ signature:
    CanonicalRankAtoms(RDKit::ROMol mol, bool breakTies=True, bool includeChirality=True, bool includeIsotopes=True)
/usr/local/lib/python3.7/dist-packages/deepchem/feat/base_classes.py:305: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.asarray(features)
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:446: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model_3/graph_pool_7/Reshape_14:0", shape=(193,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_keras_model_3/graph_pool_7/Reshape_13:0", shape=(193, 64), dtype=float32), dense_shape=Tensor("gradient_tape/pr

Executing: 5/5
############################################################


    rdkit.Chem.rdmolfiles.CanonicalRankAtoms(NoneType)
did not match C++ signature:
    CanonicalRankAtoms(RDKit::ROMol mol, bool breakTies=True, bool includeChirality=True, bool includeIsotopes=True)
/usr/local/lib/python3.7/dist-packages/deepchem/feat/base_classes.py:305: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.asarray(features)
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:446: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model_4/graph_pool_9/Reshape_14:0", shape=(193,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_keras_model_4/graph_pool_9/Reshape_13:0", shape=(193, 64), dtype=float32), dense_shape=Tensor("gradient_tape/pr

Time cost for GNN without global node: 49.114 min
Train_R2: 0.96 (+/- 0.00)
Train_RMSE: 13.25 (+/- 0.61)
Train_MAE: 8.12 (+/- 0.56)
Test_R2: 0.82 (+/- 0.02)
Test_RMSE: 28.68 (+/- 1.62)
Test_MAE: 19.35 (+/- 1.04)


In [5]:
###########################################Two Conv.##########################################
"""
graph_conv_layers: [32,32]
dense_layer_size: 128
batch_size: 20
dropout: 0.2
nb_epoch: 1500

""" 


import time

start = time.time()

metrics_rmse_train = []
metrics_mae_train = []
metrics_r2_train = []
metrics_rmse_test = []
metrics_mae_test = []
metrics_r2_test = []

for i in range(5):   # 5次循环
  print("Executing: %d/5" %(i+1))
  print("#"*60)
  os.mkdir(MODEL_DIR + "32-32" + str(i+1))
  MODEL_DIR = MODEL_DIR + "32-32" + str(i+1)
  ########################################Featurizerization#########################
  featurizer = dc.feat.ConvMolFeaturizer()
  loader = dc.data.CSVLoader(tasks=["Tg/℃"], feature_field="Smiles", featurizer=featurizer)
  dataset = loader.create_dataset(DATASET_FILE, shard_size=10000)

  splitter = dc.splits.splitters.RandomSplitter()
  trainset, testset = splitter.train_test_split(dataset, frac_train=0.8,seed=1)

  ########################################Model######################################
  model = dc.models.GraphConvModel(1, 
                  graph_conv_layers = [32,32],
                  dense_layer_size =128,
                  mode="regression",
                  batch_normalize=False,
                  batch_size=20,
                  model_dir=MODEL_DIR,
                  dropout=0.2)

  ########################################Fit########################################
  model.fit(trainset, nb_epoch=1500)

  ########################################Predict####################################
  test_pred = model.predict(testset)
  train_pred = model.predict(trainset)

  ########################################Metrics####################################
  rmse = metrics.mean_squared_error(y_true=trainset.y, y_pred=train_pred, squared=False)   # RMSE
  r2 = metrics.r2_score(y_true=trainset.y, y_pred=train_pred)
  mae = metrics.mean_absolute_error(y_true=trainset.y, y_pred=train_pred)

  rmse_test = metrics.mean_squared_error(y_true=testset.y, y_pred=test_pred, squared=False)   # RMSE
  r2_test = metrics.r2_score(y_true=testset.y, y_pred=test_pred)
  mae_test = metrics.mean_absolute_error(y_true=testset.y, y_pred=test_pred)

  metrics_r2_train.append(r2)
  metrics_rmse_train.append(rmse)
  metrics_mae_train.append(mae)
  metrics_r2_test.append(r2_test)
  metrics_rmse_test.append(rmse_test)
  metrics_mae_test.append(mae_test)

end = time.time()

print("Time cost for GNN without global node: %.3f min" % ((end-start)/60))

print("Train_R2: %.2f (+/- %.2f)" % (np.mean(metrics_r2_train), np.std(metrics_r2_train)))
print("Train_RMSE: %.2f (+/- %.2f)" % (np.mean(metrics_rmse_train), np.std(metrics_rmse_train)))
print("Train_MAE: %.2f (+/- %.2f)" % (np.mean(metrics_mae_train), np.std(metrics_mae_train)))

print("Test_R2: %.2f (+/- %.2f)" % (np.mean(metrics_r2_test), np.std(metrics_r2_test)))
print("Test_RMSE: %.2f (+/- %.2f)" % (np.mean(metrics_rmse_test), np.std(metrics_rmse_test)))
print("Test_MAE: %.2f (+/- %.2f)" % (np.mean(metrics_mae_test), np.std(metrics_mae_test)))

Executing: 1/5
############################################################


    rdkit.Chem.rdmolfiles.CanonicalRankAtoms(NoneType)
did not match C++ signature:
    CanonicalRankAtoms(RDKit::ROMol mol, bool breakTies=True, bool includeChirality=True, bool includeIsotopes=True)
/usr/local/lib/python3.7/dist-packages/deepchem/feat/base_classes.py:305: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.asarray(features)
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:446: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model_5/graph_pool_11/Reshape_14:0", shape=(193,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_keras_model_5/graph_pool_11/Reshape_13:0", shape=(193, 32), dtype=float32), dense_shape=Tensor("gradient_tape/

Executing: 2/5
############################################################


    rdkit.Chem.rdmolfiles.CanonicalRankAtoms(NoneType)
did not match C++ signature:
    CanonicalRankAtoms(RDKit::ROMol mol, bool breakTies=True, bool includeChirality=True, bool includeIsotopes=True)
/usr/local/lib/python3.7/dist-packages/deepchem/feat/base_classes.py:305: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.asarray(features)
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:446: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model_6/graph_pool_13/Reshape_14:0", shape=(193,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_keras_model_6/graph_pool_13/Reshape_13:0", shape=(193, 32), dtype=float32), dense_shape=Tensor("gradient_tape/

Executing: 3/5
############################################################


...

Executing: 4/5
############################################################


    rdkit.Chem.rdmolfiles.CanonicalRankAtoms(NoneType)
did not match C++ signature:
    CanonicalRankAtoms(RDKit::ROMol mol, bool breakTies=True, bool includeChirality=True, bool includeIsotopes=True)
/usr/local/lib/python3.7/dist-packages/deepchem/feat/base_classes.py:305: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.asarray(features)
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:446: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model_8/graph_pool_17/Reshape_14:0", shape=(193,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_keras_model_8/graph_pool_17/Reshape_13:0", shape=(193, 32), dtype=float32), dense_shape=Tensor("gradient_tape/

Executing: 5/5
############################################################


    rdkit.Chem.rdmolfiles.CanonicalRankAtoms(NoneType)
did not match C++ signature:
    CanonicalRankAtoms(RDKit::ROMol mol, bool breakTies=True, bool includeChirality=True, bool includeIsotopes=True)
/usr/local/lib/python3.7/dist-packages/deepchem/feat/base_classes.py:305: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.asarray(features)
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:446: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model_9/graph_pool_19/Reshape_14:0", shape=(193,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_keras_model_9/graph_pool_19/Reshape_13:0", shape=(193, 32), dtype=float32), dense_shape=Tensor("gradient_tape/

Time cost for GNN without global node: 38.442 min
Train_R2: 0.94 (+/- 0.01)
Train_RMSE: 14.84 (+/- 0.71)
Train_MAE: 10.17 (+/- 0.69)
Test_R2: 0.81 (+/- 0.01)
Test_RMSE: 29.19 (+/- 1.01)
Test_MAE: 19.83 (+/- 0.48)


In [6]:
###########################################Three Conv.##########################################
"""
graph_conv_layers: [64,64,64]
dense_layer_size: 128
batch_size: 20
dropout: 0.2
nb_epoch: 1500

""" 


import time

start = time.time()

metrics_rmse_train = []
metrics_mae_train = []
metrics_r2_train = []
metrics_rmse_test = []
metrics_mae_test = []
metrics_r2_test = []

for i in range(5):   # 5次循环
  print("Executing: %d/5" %(i+1))
  print("#"*60)
  os.mkdir(MODEL_DIR + "64-64-64" + str(i+1))
  MODEL_DIR = MODEL_DIR + "64-64-64" + str(i+1)
  ########################################Featurizerization#########################
  featurizer = dc.feat.ConvMolFeaturizer()
  loader = dc.data.CSVLoader(tasks=["Tg/℃"], feature_field="Smiles", featurizer=featurizer)
  dataset = loader.create_dataset(DATASET_FILE, shard_size=10000)

  splitter = dc.splits.splitters.RandomSplitter()
  trainset, testset = splitter.train_test_split(dataset, frac_train=0.8,seed=1)

  ########################################Model######################################
  model = dc.models.GraphConvModel(1, 
                  graph_conv_layers = [64,64,64],
                  dense_layer_size =128,
                  mode="regression",
                  batch_normalize=False,
                  batch_size=20,
                  model_dir=MODEL_DIR,
                  dropout=0.2)

  ########################################Fit########################################
  model.fit(trainset, nb_epoch=1500)

  ########################################Predict####################################
  test_pred = model.predict(testset)
  train_pred = model.predict(trainset)

  ########################################Metrics####################################
  rmse = metrics.mean_squared_error(y_true=trainset.y, y_pred=train_pred, squared=False)   # RMSE
  r2 = metrics.r2_score(y_true=trainset.y, y_pred=train_pred)
  mae = metrics.mean_absolute_error(y_true=trainset.y, y_pred=train_pred)

  rmse_test = metrics.mean_squared_error(y_true=testset.y, y_pred=test_pred, squared=False)   # RMSE
  r2_test = metrics.r2_score(y_true=testset.y, y_pred=test_pred)
  mae_test = metrics.mean_absolute_error(y_true=testset.y, y_pred=test_pred)

  metrics_r2_train.append(r2)
  metrics_rmse_train.append(rmse)
  metrics_mae_train.append(mae)
  metrics_r2_test.append(r2_test)
  metrics_rmse_test.append(rmse_test)
  metrics_mae_test.append(mae_test)

end = time.time()

print("Time cost for GNN without global node: %.3f min" % ((end-start)/60))

print("Train_R2: %.2f (+/- %.2f)" % (np.mean(metrics_r2_train), np.std(metrics_r2_train)))
print("Train_RMSE: %.2f (+/- %.2f)" % (np.mean(metrics_rmse_train), np.std(metrics_rmse_train)))
print("Train_MAE: %.2f (+/- %.2f)" % (np.mean(metrics_mae_train), np.std(metrics_mae_train)))

print("Test_R2: %.2f (+/- %.2f)" % (np.mean(metrics_r2_test), np.std(metrics_r2_test)))
print("Test_RMSE: %.2f (+/- %.2f)" % (np.mean(metrics_rmse_test), np.std(metrics_rmse_test)))
print("Test_MAE: %.2f (+/- %.2f)" % (np.mean(metrics_mae_test), np.std(metrics_mae_test)))

Executing: 1/5
############################################################


    rdkit.Chem.rdmolfiles.CanonicalRankAtoms(NoneType)
did not match C++ signature:
    CanonicalRankAtoms(RDKit::ROMol mol, bool breakTies=True, bool includeChirality=True, bool includeIsotopes=True)
/usr/local/lib/python3.7/dist-packages/deepchem/feat/base_classes.py:305: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.asarray(features)
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:446: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model_10/graph_pool_22/Reshape_14:0", shape=(193,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_keras_model_10/graph_pool_22/Reshape_13:0", shape=(193, 64), dtype=float32), dense_shape=Tensor("gradient_tap

Executing: 2/5
############################################################


    rdkit.Chem.rdmolfiles.CanonicalRankAtoms(NoneType)
did not match C++ signature:
    CanonicalRankAtoms(RDKit::ROMol mol, bool breakTies=True, bool includeChirality=True, bool includeIsotopes=True)
/usr/local/lib/python3.7/dist-packages/deepchem/feat/base_classes.py:305: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.asarray(features)
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:446: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model_11/graph_pool_25/Reshape_14:0", shape=(193,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_keras_model_11/graph_pool_25/Reshape_13:0", shape=(193, 64), dtype=float32), dense_shape=Tensor("gradient_tap

Executing: 3/5
############################################################


...

Executing: 4/5
############################################################


    rdkit.Chem.rdmolfiles.CanonicalRankAtoms(NoneType)
did not match C++ signature:
    CanonicalRankAtoms(RDKit::ROMol mol, bool breakTies=True, bool includeChirality=True, bool includeIsotopes=True)
/usr/local/lib/python3.7/dist-packages/deepchem/feat/base_classes.py:305: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.asarray(features)
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:446: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model_13/graph_pool_31/Reshape_14:0", shape=(193,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_keras_model_13/graph_pool_31/Reshape_13:0", shape=(193, 64), dtype=float32), dense_shape=Tensor("gradient_tap

Executing: 5/5
############################################################


    rdkit.Chem.rdmolfiles.CanonicalRankAtoms(NoneType)
did not match C++ signature:
    CanonicalRankAtoms(RDKit::ROMol mol, bool breakTies=True, bool includeChirality=True, bool includeIsotopes=True)
/usr/local/lib/python3.7/dist-packages/deepchem/feat/base_classes.py:305: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.asarray(features)
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:446: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model_14/graph_pool_34/Reshape_14:0", shape=(193,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_keras_model_14/graph_pool_34/Reshape_13:0", shape=(193, 64), dtype=float32), dense_shape=Tensor("gradient_tap

Time cost for GNN without global node: 65.130 min
Train_R2: 0.87 (+/- 0.04)
Train_RMSE: 22.76 (+/- 3.73)
Train_MAE: 13.07 (+/- 2.31)
Test_R2: 0.72 (+/- 0.06)
Test_RMSE: 35.71 (+/- 3.91)
Test_MAE: 22.84 (+/- 2.79)


In [8]:
###########################################Two Conv.##########################################
"""
graph_conv_layers: [64,64]
dense_layer_size: 64
batch_size: 20
dropout: 0.2
nb_epoch: 1500

""" 


import time

start = time.time()

metrics_rmse_train = []
metrics_mae_train = []
metrics_r2_train = []
metrics_rmse_test = []
metrics_mae_test = []
metrics_r2_test = []

for i in range(5):   # 5次循环
  print("Executing: %d/5" %(i+1))
  print("#"*60)
  os.mkdir(MODEL_DIR + "64-64+128" + str(i+1))
  MODEL_DIR = MODEL_DIR + "64-64+128" + str(i+1)
  ########################################Featurizerization#########################
  featurizer = dc.feat.ConvMolFeaturizer()
  loader = dc.data.CSVLoader(tasks=["Tg/℃"], feature_field="Smiles", featurizer=featurizer)
  dataset = loader.create_dataset(DATASET_FILE, shard_size=10000)

  splitter = dc.splits.splitters.RandomSplitter()
  trainset, testset = splitter.train_test_split(dataset, frac_train=0.8,seed=1)

  ########################################Model######################################
  model = dc.models.GraphConvModel(1, 
                  dense_layer_size = 64,
                  mode="regression",
                  batch_normalize=False,
                  batch_size=20,
                  model_dir=MODEL_DIR,
                  dropout=0.2)

  ########################################Fit########################################
  model.fit(trainset, nb_epoch=1500)

  ########################################Predict####################################
  test_pred = model.predict(testset)
  train_pred = model.predict(trainset)

  ########################################Metrics####################################
  rmse = metrics.mean_squared_error(y_true=trainset.y, y_pred=train_pred, squared=False)   # RMSE
  r2 = metrics.r2_score(y_true=trainset.y, y_pred=train_pred)
  mae = metrics.mean_absolute_error(y_true=trainset.y, y_pred=train_pred)

  rmse_test = metrics.mean_squared_error(y_true=testset.y, y_pred=test_pred, squared=False)   # RMSE
  r2_test = metrics.r2_score(y_true=testset.y, y_pred=test_pred)
  mae_test = metrics.mean_absolute_error(y_true=testset.y, y_pred=test_pred)

  metrics_r2_train.append(r2)
  metrics_rmse_train.append(rmse)
  metrics_mae_train.append(mae)
  metrics_r2_test.append(r2_test)
  metrics_rmse_test.append(rmse_test)
  metrics_mae_test.append(mae_test)

end = time.time()

print("Time cost for GNN without global node: %.3f min" % ((end-start)/60))

print("Train_R2: %.2f (+/- %.2f)" % (np.mean(metrics_r2_train), np.std(metrics_r2_train)))
print("Train_RMSE: %.2f (+/- %.2f)" % (np.mean(metrics_rmse_train), np.std(metrics_rmse_train)))
print("Train_MAE: %.2f (+/- %.2f)" % (np.mean(metrics_mae_train), np.std(metrics_mae_train)))

print("Test_R2: %.2f (+/- %.2f)" % (np.mean(metrics_r2_test), np.std(metrics_r2_test)))
print("Test_RMSE: %.2f (+/- %.2f)" % (np.mean(metrics_rmse_test), np.std(metrics_rmse_test)))
print("Test_MAE: %.2f (+/- %.2f)" % (np.mean(metrics_mae_test), np.std(metrics_mae_test)))

Executing: 1/5
############################################################


    rdkit.Chem.rdmolfiles.CanonicalRankAtoms(NoneType)
did not match C++ signature:
    CanonicalRankAtoms(RDKit::ROMol mol, bool breakTies=True, bool includeChirality=True, bool includeIsotopes=True)
/usr/local/lib/python3.7/dist-packages/deepchem/feat/base_classes.py:305: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.asarray(features)
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:446: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model_15/graph_pool_36/Reshape_14:0", shape=(193,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_keras_model_15/graph_pool_36/Reshape_13:0", shape=(193, 64), dtype=float32), dense_shape=Tensor("gradient_tap

Executing: 2/5
############################################################


    rdkit.Chem.rdmolfiles.CanonicalRankAtoms(NoneType)
did not match C++ signature:
    CanonicalRankAtoms(RDKit::ROMol mol, bool breakTies=True, bool includeChirality=True, bool includeIsotopes=True)
/usr/local/lib/python3.7/dist-packages/deepchem/feat/base_classes.py:305: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.asarray(features)
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:446: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model_16/graph_pool_38/Reshape_14:0", shape=(193,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_keras_model_16/graph_pool_38/Reshape_13:0", shape=(193, 64), dtype=float32), dense_shape=Tensor("gradient_tap

Executing: 3/5
############################################################


...

Executing: 4/5
############################################################


    rdkit.Chem.rdmolfiles.CanonicalRankAtoms(NoneType)
did not match C++ signature:
    CanonicalRankAtoms(RDKit::ROMol mol, bool breakTies=True, bool includeChirality=True, bool includeIsotopes=True)
/usr/local/lib/python3.7/dist-packages/deepchem/feat/base_classes.py:305: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.asarray(features)
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:446: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model_18/graph_pool_42/Reshape_14:0", shape=(193,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_keras_model_18/graph_pool_42/Reshape_13:0", shape=(193, 64), dtype=float32), dense_shape=Tensor("gradient_tap

Executing: 5/5
############################################################


    rdkit.Chem.rdmolfiles.CanonicalRankAtoms(NoneType)
did not match C++ signature:
    CanonicalRankAtoms(RDKit::ROMol mol, bool breakTies=True, bool includeChirality=True, bool includeIsotopes=True)
/usr/local/lib/python3.7/dist-packages/deepchem/feat/base_classes.py:305: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.asarray(features)
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:446: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model_19/graph_pool_44/Reshape_14:0", shape=(193,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_keras_model_19/graph_pool_44/Reshape_13:0", shape=(193, 64), dtype=float32), dense_shape=Tensor("gradient_tap

Time cost for GNN without global node: 48.424 min
Train_R2: 0.92 (+/- 0.00)
Train_RMSE: 17.82 (+/- 0.32)
Train_MAE: 9.44 (+/- 0.39)
Test_R2: 0.79 (+/- 0.04)
Test_RMSE: 31.13 (+/- 3.17)
Test_MAE: 20.58 (+/- 1.81)


In [ ]:
# default, shard_size=10000   1层卷积[64] dense_layer_size = 64
featurizer = dc.feat.ConvMolFeaturizer(master_atom=True)
loader = dc.data.CSVLoader(tasks=["Tg/℃"], feature_field="Smiles", featurizer=featurizer)
dataset = loader.create_dataset(DATASET_FILE, shard_size=10000)

splitter = dc.splits.splitters.RandomSplitter()
trainset, testset = splitter.train_test_split(dataset, frac_train=0.8,seed=1)

model = dc.models.GraphConvModel(
    1, 
    graph_conv_layers = [32], dense_layer_size = 64, number_atom_features=100,
    mode="regression", batch_normalize=False, batch_size=20,model_dir=MODEL_DIR, dropout=0.1)
# model = dc.models.DAGModel(1, mode="regression", batch_normalize=False, batch_size=10,model_dir=MODEL_DIR, dropout=0.2)

model.fit(trainset, nb_epoch=1500)

test_pred = model.predict(testset)

mse = metrics.mean_squared_error(y_true=testset.y, y_pred=test_pred)
r2 = metrics.r2_score(y_true=testset.y, y_pred=test_pred)
mae = metrics.mean_absolute_error(y_true=testset.y, y_pred=test_pred)
mse,mae, r2 

    rdkit.Chem.rdmolfiles.CanonicalRankAtoms(NoneType)
did not match C++ signature:
    CanonicalRankAtoms(RDKit::ROMol mol, bool breakTies=True, bool includeChirality=True, bool includeIsotopes=True)
/usr/local/lib/python3.7/dist-packages/deepchem/feat/base_classes.py:305: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.asarray(features)
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:446: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model_5/graph_pool_8/Reshape_14:0", shape=(386,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_keras_model_5/graph_pool_8/Reshape_13:0", shape=(386, 32), dtype=float32), dense_shape=Tensor("gradient_tape/pr

(1177.5813017209118, 21.54609590657552, 0.7421948118455424)

In [ ]:
TESTSET_FILE ='/content/drive/My Drive/DL/20220902-count372/候选数据库/1-1.csv'

In [ ]:
featurizer = dc.feat.ConvMolFeaturizer()
loader = dc.data.CSVLoader(tasks=[], feature_field="Smiles", featurizer=featurizer)
testset = loader.create_dataset(TESTSET_FILE, shard_size=10000)

In [ ]:
MODEL_DIR = '/content/drive/My Drive/DL/GNN_model'

m = dc.models.GraphConvModel(1, mode="regression", model_dir=MODEL_DIR)
m.restore()
pred = m.predict(testset)

import pandas as pd 
df = pd.DataFrame(pred)
df.columns=['pred']
df.head()
df.to_csv('/content/drive/My Drive/DL/20220902-count372/预测结果/1-1.csv')

In [ ]:

# default, shard_size=20000
featurizer = dc.feat.ConvMolFeaturizer()
loader = dc.data.CSVLoader(tasks=["Tg/℃"], feature_field="Smiles", featurizer=featurizer)
dataset = loader.create_dataset(DATASET_FILE, shard_size=20000)

splitter = dc.splits.splitters.RandomSplitter()
trainset, testset = splitter.train_test_split(dataset, frac_train=0.8,seed=1)

model = dc.models.GraphConvModel(
    1, mode="regression", batch_normalize=False, batch_size=20,model_dir=MODEL_DIR, dropout=0.2)
# model = dc.models.DAGModel(1, mode="regression", batch_normalize=False, batch_size=10,model_dir=MODEL_DIR, dropout=0.2)

model.fit(trainset, nb_epoch=2000)

test_pred = model.predict(testset)

mse = metrics.mean_squared_error(y_true=testset.y, y_pred=test_pred)
r2 = metrics.r2_score(y_true=testset.y, y_pred=test_pred)
mae = metrics.mean_absolute_error(y_true=testset.y, y_pred=test_pred)
mse,mae, r2 

In [ ]:

# default, dropout=0.1
featurizer = dc.feat.ConvMolFeaturizer()
loader = dc.data.CSVLoader(tasks=["Tg/℃"], feature_field="Smiles", featurizer=featurizer)
dataset = loader.create_dataset(DATASET_FILE, shard_size=10000)

splitter = dc.splits.splitters.RandomSplitter()
trainset, testset = splitter.train_test_split(dataset, frac_train=0.8,seed=1)

model = dc.models.GraphConvModel(
    1, mode="regression", batch_normalize=False, batch_size=20,model_dir=MODEL_DIR, dropout=0.1)
# model = dc.models.DAGModel(1, mode="regression", batch_normalize=False, batch_size=10,model_dir=MODEL_DIR, dropout=0.2)

model.fit(trainset, nb_epoch=1500)

test_pred = model.predict(testset)

mse = metrics.mean_squared_error(y_true=testset.y, y_pred=test_pred)
r2 = metrics.r2_score(y_true=testset.y, y_pred=test_pred)
mae = metrics.mean_absolute_error(y_true=testset.y, y_pred=test_pred)
mse,mae, r2 